In [1]:
import csv
from datetime import datetime
import requests
from bs4 import BeautifulSoup

In [2]:
def get_url(position, location):
    # generate a url from position and location

    template = 'https://indeed.com/jobs?q={}&l={}'
    url = template.format(position, location)
    return url

In [3]:
def get_record(card):
    atag = card.h2.span
    job_title = (card.find('h2', 'jobTitle').text.strip()).lstrip('new')
    company = card.find('span','companyName').text.strip()
    job_location = card.find('div','companyLocation').text.strip()
    job_summary = card.find('div','job-snippet').text.strip()
    post_date = (card.find('span','date').text.strip()).lstrip('Posted')
    today = datetime.today().strftime('%Y-%m-%d')
    try:
        job_salary = card.find('div','salary-snippet').text.strip()
    except AttributeError:
        job_salary = ''

    record = (job_title, company, job_location, post_date, today, job_summary, job_salary)

    return record

In [4]:
url = get_url('data science', 'Toronto')

response = requests.get(url)

soup = BeautifulSoup(response.text, 'html.parser')

cards = soup.find_all('div', 'job_seen_beacon')

In [5]:
card = cards[6]
atag = card.h2.span

In [6]:
print((card.find('h2', 'jobTitle').text.strip()).lstrip('new'))

Cognitive Neuroscientist (R&D)


In [7]:
print(card.find('span','companyName').text.strip())

Intheon


In [8]:
card = cards[0]

In [9]:
get_record(card)

('Data Analyst',
 'Excel Impact',
 'Ohio',
 '30+ days ago',
 '2022-05-08',
 'Our proprietary ImpactNavigation® platform technology utilizes machine learning, data science, and intelligent-marketing to guide & match interested customers…',
 '')

In [10]:
def main(position, location):

    records = []
    url= get_url(position,location)

    while True:
        response = requests.get(url)
        soup = BeautifulSoup(response.text, 'html.parser')
        cards = soup.find_all('div', 'job_seen_beacon')

        for card in cards:
            record = get_record(card)
            records.append(record)

        try:
            url = 'https://indeed.com/' + (soup.find('a', {'aria-label': 'Next'}).get('href'))
        except AttributeError:
            break
            
    with open('job_postings_Toronto.csv', 'w', newline='',encoding='utf-8') as f:
        writer = csv.writer(f)
        writer.writerow(['job_title', 'company', 'job_location', 'post_date','extract_date','job_description', 'job_salary'])
        writer.writerows(records)

In [ ]:
main('data science','Toronto')